In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
# Dummy data
titles = ["Help with order", "Issue with payment", "Login problem"]
bodies = ["I cannot find my order", "Payment method not working", "Cannot log in to my account"]
tags = [0, 1, 2]  # categorical tags

# Tokenize the text inputs
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(titles + bodies)
title_sequences = tokenizer.texts_to_sequences(titles)
body_sequences = tokenizer.texts_to_sequences(bodies)

# Pad the sequences
title_data = pad_sequences(title_sequences, maxlen=5)
body_data = pad_sequences(body_sequences, maxlen=20)

In [7]:
# One-hot encode the tags
tag_data = to_categorical(tags, num_classes=3)

# Dummy outputs
priority_scores = np.random.rand(len(titles), 1)
departments = np.random.randint(0, 3, len(titles))
department_data = to_categorical(departments, num_classes=3)

print(priority_scores)
print(departments)
print(department_data)

[[0.88623702]
 [0.56034368]
 [0.8093288 ]]
[0 1 1]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [8]:
# Model inputs
title_input = Input(shape=(5,), name='title_input')
body_input = Input(shape=(20,), name='body_input')
tag_input = Input(shape=(3,), name='tag_input')

# Embedding and LSTM layers for text inputs
embedding_layer = Embedding(input_dim=100, output_dim=10, input_length=5)
title_embeddings = embedding_layer(title_input)
title_lstm = LSTM(32)(title_embeddings)

embedding_layer_body = Embedding(input_dim=100, output_dim=10, input_length=20)
body_embeddings = embedding_layer_body(body_input)
body_lstm = LSTM(32)(body_embeddings)

c:\Users\ppawa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
# Concatenate all inputs
concat_layer = concatenate([title_lstm, body_lstm, tag_input])

# Dense layers for output
dense1 = Dense(64, activation='relu')(concat_layer)
priority_output = Dense(1, 
                        activation='sigmoid', 
                        name='priority_output')(dense1)
department_output = Dense(3, 
                          activation='softmax', 
                          name='department_output')(dense1)

# Model definition
model = Model(inputs=[title_input, body_input, tag_input], 
              outputs=[priority_output, department_output])

In [10]:
# Compile the model
model.compile(optimizer='adam', 
              loss={'priority_output': 'binary_crossentropy', 
                    'department_output': 'categorical_crossentropy'},
              metrics={'priority_output': 'accuracy', 
                       'department_output': 'accuracy'})

In [12]:
# Train the model on dummy data
model.fit([title_data, body_data, tag_data], 
          [priority_scores, department_data], 
          epochs=45, 
          batch_size=2)

Epoch 1/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - department_output_accuracy: 1.0000 - loss: 0.5384 - priority_output_accuracy: 0.0000e+00 
Epoch 2/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - department_output_accuracy: 1.0000 - loss: 0.5383 - priority_output_accuracy: 0.0000e+00 
Epoch 3/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - department_output_accuracy: 1.0000 - loss: 0.5164 - priority_output_accuracy: 0.0000e+00 
Epoch 4/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - department_output_accuracy: 1.0000 - loss: 0.5161 - priority_output_accuracy: 0.0000e+00 
Epoch 5/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - department_output_accuracy: 1.0000 - loss: 0.5160 - priority_output_accuracy: 0.0000e+00 
Epoch 6/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - department_output_accuracy: 1.0000 - loss: 0.5160 - priority_output_accuracy: 0.0000e+00 
Epoch 7/45
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - department_output_accuracy: 1.0000 - loss: 0.4828 - priority_output_accuracy: 0.0000e+00
Epoch 8/45
2/2 ━━━━━